In [5]:
import csv

In [1]:
from mach_helper.benchmark import Benchmark

In [11]:
hc_dataset = []
with open('/Users/zhengyanglumacmini/Desktop/Projects/lm_smt/bin_sage2/raw/merged_3_mini_train.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        path = row[0]
        print(path)
        benchmark = Benchmark(path)
        benchmark.parse()
        feature = benchmark.features
        assert len(feature) == 189
        z3_label = 0
        if row[1] == 'True':
            z3_label = 1
        feature.append(z3_label)
        hc_dataset.append(feature)

/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_6360.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_550.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_12386.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_16173.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_1447.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_8506.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_14058.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_10403.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_11072.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_7397.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_8732.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_8246.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_10955.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_9946.smt2
/Users/zhengyan

In [12]:
with open('/Users/zhengyanglumacmini/Desktop/Projects/lm_smt/bin_sage2/z3_mach_train.csv', 'w') as f:
    writer = csv.writer(f)
    # creat a list of column names, f0, f1, ..., f188, z3_label
    columns = ['f' + str(i) for i in range(189)]
    columns.append('z3_label')
    writer.writerow(columns)
    for row in hc_dataset:
        writer.writerow(row)


In [13]:
hc_test_dataset = []
with open('/Users/zhengyanglumacmini/Desktop/Projects/lm_smt/bin_sage2/raw/merged_3_mini_test.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        path = row[0]
        print(path)
        benchmark = Benchmark(path)
        benchmark.parse()
        feature = benchmark.features
        assert len(feature) == 189
        z3_label = 0
        if row[1] == 'True':
            z3_label = 1
        feature.append(z3_label)
        hc_test_dataset.append(feature)

/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_7148.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_8248.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_10233.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_3162.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_1056.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_16996.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_2915.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_1590.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_9516.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_16438.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_13569.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_2326.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_5701.smt2
/Users/zhengyanglumacmini/Desktop/smt-lib/QF_BV/Sage2/bench_3022.smt2
/Users/zhengyang

In [14]:
with open('/Users/zhengyanglumacmini/Desktop/Projects/lm_smt/bin_sage2/z3_mach_test.csv', 'w') as f:
    writer = csv.writer(f)
    # creat a list of column names, f0, f1, ..., f188, z3_label
    columns = ['f' + str(i) for i in range(189)]
    columns.append('z3_label')
    writer.writerow(columns)
    for row in hc_test_dataset:
        writer.writerow(row)

In [15]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [16]:
def load_data(file_path):
    data = pd.read_csv(file_path)
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    return X, y

In [18]:
X_train, y_train = load_data('/Users/zhengyanglumacmini/Desktop/Projects/lm_smt/bin_sage2/z3_mach_train.csv')
X_test, y_test = load_data('/Users/zhengyanglumacmini/Desktop/Projects/lm_smt/bin_sage2/z3_mach_test.csv')

In [19]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [20]:
X_train, y_train = torch.tensor(X_train).float(), torch.tensor(y_train).float()
X_test, y_test = torch.tensor(X_test).float(), torch.tensor(y_test).float()

In [21]:
class SolverClassifyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return len(self.y)

In [23]:
train_dataset = SolverClassifyDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [24]:
test_dataset = SolverClassifyDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [25]:
class BinaryClassificationModel(nn.Module):
    def __init__(self):
        super(BinaryClassificationModel, self).__init__()
        self.layer1 = nn.Linear(X_train.shape[1], 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = torch.sigmoid(self.layer3(x))
        return x

In [26]:
model = BinaryClassificationModel()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [28]:
for epoch in range(100):
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
    print('Epoch: {}, Loss: {:.4f}'.format(epoch, loss.item()))

Epoch: 0, Loss: 0.5689
Epoch: 1, Loss: 0.1473
Epoch: 2, Loss: 0.1896
Epoch: 3, Loss: 0.3299
Epoch: 4, Loss: 0.0853
Epoch: 5, Loss: 0.1209
Epoch: 6, Loss: 0.3179
Epoch: 7, Loss: 0.4393
Epoch: 8, Loss: 0.1566
Epoch: 9, Loss: 0.0836
Epoch: 10, Loss: 0.2779
Epoch: 11, Loss: 0.0258
Epoch: 12, Loss: 0.1844
Epoch: 13, Loss: 0.2566
Epoch: 14, Loss: 0.2358
Epoch: 15, Loss: 0.1195
Epoch: 16, Loss: 0.1846
Epoch: 17, Loss: 0.0580
Epoch: 18, Loss: 0.4106
Epoch: 19, Loss: 0.0915
Epoch: 20, Loss: 0.4499
Epoch: 21, Loss: 0.5128
Epoch: 22, Loss: 0.1813
Epoch: 23, Loss: 0.6532
Epoch: 24, Loss: 0.2569
Epoch: 25, Loss: 0.1800
Epoch: 26, Loss: 0.1596
Epoch: 27, Loss: 0.2016
Epoch: 28, Loss: 0.0998
Epoch: 29, Loss: 0.8184
Epoch: 30, Loss: 0.2480
Epoch: 31, Loss: 0.2352
Epoch: 32, Loss: 0.2583
Epoch: 33, Loss: 0.0741
Epoch: 34, Loss: 0.1399
Epoch: 35, Loss: 0.1067
Epoch: 36, Loss: 0.2240
Epoch: 37, Loss: 0.2013
Epoch: 38, Loss: 0.1887
Epoch: 39, Loss: 0.0294
Epoch: 40, Loss: 0.1041
Epoch: 41, Loss: 0.3701
Ep

In [37]:
model.eval()
preds = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        # preds is a list of 0 and 1
        preds += list(torch.round(outputs.squeeze()).numpy())
    

In [43]:
true_labels = list(y_test.numpy())

In [44]:
with open('./z3_mach_preds.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['preds', 'labels'])
    for i in range(len(preds)):
        writer.writerow([int(preds[i]), int(true_labels[i])])